# Week 1, Lab 2: Informed Search

## Welcome Back!

In Lab 1, we explored **uninformed search** algorithms (DFS and BFS) that blindly explore the search space. Today, we'll learn **informed search** algorithms that use domain knowledge to find solutions more efficiently.

### What You'll Learn

- Uniform Cost Search (UCS)
- Greedy Best-First Search
- A* Search (the gold standard!)
- Heuristic functions and admissibility
- When to use each algorithm

### Key Insight

**Informed search uses heuristics** - estimates of how close we are to the goal. This makes search much more efficient!

Think of it like:
- **Uninformed**: Searching for your keys by checking every room
- **Informed**: Checking rooms where you last remember having them

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from heapq import heappush, heappop
from typing import List, Tuple, Set, Optional, Callable
import time

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Path Cost: A New Dimension

So far, we've only cared about **finding a path**. But in the real world, some paths are better than others!

Example: GPS navigation
- Route A: 10 miles, highway (fast)
- Route B: 8 miles, city streets (slow)

Which is better? It depends on the **cost** (time, distance, fuel, etc.)!

In [ ]:
# Create a weighted maze where different terrains have different costs
# 0 = open (cost 1), 1 = wall (can't pass), 2 = rough terrain (cost 3)
weighted_maze = np.array([
    [0, 0, 2, 0, 0],
    [0, 1, 2, 0, 0],
    [0, 0, 0, 2, 1],
    [1, 1, 0, 1, 0],
    [0, 0, 0, 0, 0]
])

# Map terrain types to costs
TERRAIN_COSTS = {0: 1, 2: 3}

start = (0, 0)
goal = (4, 4)

def visualize_weighted_maze(maze, start, goal, path=None):
    """Visualize maze with different terrain costs."""
    plt.figure(figsize=(10, 8))
    
    # Create display with colors for different terrains
    display = maze.astype(float)
    display[maze == 0] = 0.2  # Open terrain (light)
    display[maze == 1] = 1.0  # Walls (dark)
    display[maze == 2] = 0.5  # Rough terrain (medium)
    
    # Mark path if provided
    if path:
        for pos in path[1:-1]:
            display[pos] = 0.35
    
    plt.imshow(display, cmap='RdYlGn_r', interpolation='nearest')
    plt.colorbar(label='Terrain Type', ticks=[0.2, 0.5, 1.0], 
                format=plt.FuncFormatter(lambda x, p: ['Open(1)', 'Rough(3)', 'Wall'][p]))
    
    # Add grid
    for i in range(maze.shape[0] + 1):
        plt.axhline(i - 0.5, color='black', linewidth=0.5)
    for j in range(maze.shape[1] + 1):
        plt.axvline(j - 0.5, color='black', linewidth=0.5)
    
    # Labels
    plt.text(start[1], start[0], 'S', ha='center', va='center',
             fontsize=20, fontweight='bold', color='blue')
    plt.text(goal[1], goal[0], 'G', ha='center', va='center',
             fontsize=20, fontweight='bold', color='green')
    
    # Add cost labels
    for i in range(maze.shape[0]):
        for j in range(maze.shape[1]):
            if maze[i, j] != 1:  # Not a wall
                cost = TERRAIN_COSTS[maze[i, j]]
                if (i, j) != start and (i, j) != goal:
                    plt.text(j, i, str(cost), ha='center', va='center',
                           fontsize=10, color='gray', alpha=0.7)
    
    plt.title('Weighted Maze: Numbers show movement cost')
    plt.tight_layout()
    plt.show()

visualize_weighted_maze(weighted_maze, start, goal)

In [ ]:
# Helper function for weighted neighbors
def get_weighted_neighbors(maze: np.ndarray, pos: Tuple[int, int]) -> List[Tuple[Tuple[int, int], int]]:
    """Get valid neighbors with their movement costs."""
    rows, cols = maze.shape
    row, col = pos
    neighbors = []
    
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    for dr, dc in directions:
        new_row, new_col = row + dr, col + dc
        
        if (0 <= new_row < rows and 
            0 <= new_col < cols and 
            maze[new_row, new_col] != 1):  # Not a wall
            neighbor_pos = (new_row, new_col)
            cost = TERRAIN_COSTS[maze[new_row, new_col]]
            neighbors.append((neighbor_pos, cost))
    
    return neighbors

# Test
test_pos = (0, 1)
neighbors = get_weighted_neighbors(weighted_maze, test_pos)
print(f"Neighbors of {test_pos}:")
for pos, cost in neighbors:
    print(f"  {pos} with cost {cost}")

## 2. Uniform Cost Search (UCS)

UCS is like BFS but considers path cost!

### How it works:
1. Always expand the **lowest-cost** path first
2. Use a **priority queue** (min-heap)
3. Guaranteed to find the **optimal (lowest-cost) path**

### Key Difference from BFS:
- **BFS**: Shortest path (fewest steps)
- **UCS**: Cheapest path (lowest total cost)

In [ ]:
def uniform_cost_search(maze: np.ndarray, start: Tuple[int, int], goal: Tuple[int, int]):
    """
    Uniform Cost Search - finds the lowest-cost path.
    
    Returns:
        (path, total_cost, nodes_explored)
    """
    # Priority queue: (cost, counter, current, path)
    counter = 0  # For tie-breaking
    pq = [(0, counter, start, [start])]
    visited = set()
    nodes_explored = 0
    
    while pq:
        cost, _, current, path = heappop(pq)
        
        if current in visited:
            continue
        
        visited.add(current)
        nodes_explored += 1
        
        # Check if goal
        if current == goal:
            print(f"✓ UCS found optimal path! Cost: {cost}, Explored: {nodes_explored} nodes")
            return path, cost, nodes_explored
        
        # Explore neighbors
        for neighbor, step_cost in get_weighted_neighbors(maze, current):
            if neighbor not in visited:
                new_cost = cost + step_cost
                new_path = path + [neighbor]
                counter += 1
                heappush(pq, (new_cost, counter, neighbor, new_path))
    
    print(f"✗ UCS found no path")
    return None, float('inf'), nodes_explored

# Run UCS
ucs_path, ucs_cost, ucs_nodes = uniform_cost_search(weighted_maze, start, goal)

if ucs_path:
    print(f"\nPath length: {len(ucs_path)} steps")
    print(f"Total cost: {ucs_cost}")
    visualize_weighted_maze(weighted_maze, start, goal, ucs_path)

## 3. Heuristic Functions

A **heuristic** is an estimate of the cost from a position to the goal.

### Common Heuristics for Grid Mazes:

1. **Manhattan Distance** (L1): Sum of horizontal and vertical distances
   - `h(n) = |x1 - x2| + |y1 - y2|`
   - Works for 4-directional movement

2. **Euclidean Distance** (L2): Straight-line distance
   - `h(n) = sqrt((x1-x2)² + (y1-y2)²)`
   - Works for diagonal or any-direction movement

3. **Chebyshev Distance**: Maximum of horizontal and vertical distances
   - `h(n) = max(|x1 - x2|, |y1 - y2|)`
   - Works for 8-directional movement

In [ ]:
def manhattan_distance(pos1: Tuple[int, int], pos2: Tuple[int, int]) -> float:
    """Manhattan distance heuristic (L1 norm)."""
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

def euclidean_distance(pos1: Tuple[int, int], pos2: Tuple[int, int]) -> float:
    """Euclidean distance heuristic (L2 norm)."""
    return np.sqrt((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)

def chebyshev_distance(pos1: Tuple[int, int], pos2: Tuple[int, int]) -> float:
    """Chebyshev distance heuristic (L-infinity norm)."""
    return max(abs(pos1[0] - pos2[0]), abs(pos1[1] - pos2[1]))

# Visualize heuristics
test_pos = (1, 1)
goal_pos = (4, 4)

print(f"From {test_pos} to {goal_pos}:")
print(f"  Manhattan: {manhattan_distance(test_pos, goal_pos)}")
print(f"  Euclidean: {euclidean_distance(test_pos, goal_pos):.2f}")
print(f"  Chebyshev: {chebyshev_distance(test_pos, goal_pos)}")

# Visualize heuristic values across the maze
def visualize_heuristic(maze, goal, heuristic_func, title):
    """Show heuristic values for all positions."""
    fig, ax = plt.subplots(figsize=(10, 8))
    
    heuristic_values = np.zeros_like(maze, dtype=float)
    
    for i in range(maze.shape[0]):
        for j in range(maze.shape[1]):
            if maze[i, j] != 1:  # Not a wall
                heuristic_values[i, j] = heuristic_func((i, j), goal)
            else:
                heuristic_values[i, j] = np.nan
    
    im = ax.imshow(heuristic_values, cmap='YlOrRd', interpolation='nearest')
    plt.colorbar(im, label='Heuristic Value (distance to goal)')
    
    # Add values as text
    for i in range(maze.shape[0]):
        for j in range(maze.shape[1]):
            if maze[i, j] != 1:
                text = ax.text(j, i, f'{heuristic_values[i, j]:.1f}',
                             ha='center', va='center', color='black', fontsize=10)
    
    ax.text(goal[1], goal[0], 'G', ha='center', va='center',
           fontsize=20, fontweight='bold', color='green')
    
    ax.set_title(f'{title} - Lower values = closer to goal')
    plt.tight_layout()
    plt.show()

visualize_heuristic(weighted_maze, goal, manhattan_distance, 'Manhattan Distance Heuristic')

## 4. Greedy Best-First Search

Greedy search always moves toward the position that **looks closest to the goal** (lowest heuristic value).

### How it works:
1. Expand the node with the **lowest heuristic value** h(n)
2. Ignores the cost to get there!
3. Fast but **not guaranteed to find optimal path**

### Analogy:
Like following a compass directly toward your destination without considering obstacles or terrain.

In [ ]:
def greedy_search(maze: np.ndarray, start: Tuple[int, int], goal: Tuple[int, int], 
                  heuristic: Callable = manhattan_distance):
    """
    Greedy Best-First Search - follows the heuristic greedily.
    Fast but not optimal.
    
    Returns:
        (path, total_cost, nodes_explored)
    """
    counter = 0
    # Priority queue: (heuristic, counter, current, path, cost)
    pq = [(heuristic(start, goal), counter, start, [start], 0)]
    visited = set()
    nodes_explored = 0
    
    while pq:
        h, _, current, path, cost = heappop(pq)
        
        if current in visited:
            continue
        
        visited.add(current)
        nodes_explored += 1
        
        if current == goal:
            print(f"✓ Greedy found a path! Cost: {cost}, Explored: {nodes_explored} nodes")
            return path, cost, nodes_explored
        
        for neighbor, step_cost in get_weighted_neighbors(maze, current):
            if neighbor not in visited:
                h_value = heuristic(neighbor, goal)
                new_cost = cost + step_cost
                new_path = path + [neighbor]
                counter += 1
                heappush(pq, (h_value, counter, neighbor, new_path, new_cost))
    
    print(f"✗ Greedy found no path")
    return None, float('inf'), nodes_explored

# Run Greedy
greedy_path, greedy_cost, greedy_nodes = greedy_search(weighted_maze, start, goal)

if greedy_path:
    print(f"\nPath length: {len(greedy_path)} steps")
    print(f"Total cost: {greedy_cost}")
    visualize_weighted_maze(weighted_maze, start, goal, greedy_path)

## 5. A* Search - The Best of Both Worlds! ⭐

A* combines the best aspects of UCS and Greedy:
- Like UCS: Considers actual cost so far (g)
- Like Greedy: Uses heuristic to estimate remaining cost (h)

### The A* Formula:
```
f(n) = g(n) + h(n)
```
Where:
- **g(n)** = actual cost from start to n
- **h(n)** = estimated cost from n to goal (heuristic)
- **f(n)** = estimated total cost of path through n

### Why A* is Amazing:
1. **Optimal**: Finds lowest-cost path (if heuristic is admissible)
2. **Efficient**: Often explores fewer nodes than UCS
3. **Complete**: Will find a solution if one exists

### Admissible Heuristic:
A heuristic is **admissible** if it **never overestimates** the true cost to the goal.
- Manhattan distance is admissible for 4-directional movement
- Euclidean distance is admissible for any-direction movement

In [ ]:
def a_star(maze: np.ndarray, start: Tuple[int, int], goal: Tuple[int, int],
           heuristic: Callable = manhattan_distance):
    """
    A* Search - optimal and efficient!
    
    Uses f(n) = g(n) + h(n) where:
    - g(n) = actual cost from start
    - h(n) = estimated cost to goal
    
    Returns:
        (path, total_cost, nodes_explored)
    """
    counter = 0
    # Priority queue: (f_score, counter, current, path, g_score)
    h_start = heuristic(start, goal)
    pq = [(h_start, counter, start, [start], 0)]
    visited = set()
    nodes_explored = 0
    
    # For visualization: track f and g scores
    g_scores = {start: 0}
    
    while pq:
        f, _, current, path, g = heappop(pq)
        
        if current in visited:
            continue
        
        visited.add(current)
        nodes_explored += 1
        
        if current == goal:
            print(f"✓ A* found optimal path! Cost: {g}, Explored: {nodes_explored} nodes")
            return path, g, nodes_explored
        
        for neighbor, step_cost in get_weighted_neighbors(maze, current):
            if neighbor not in visited:
                new_g = g + step_cost
                
                # Only process if this is a better path to neighbor
                if neighbor not in g_scores or new_g < g_scores[neighbor]:
                    g_scores[neighbor] = new_g
                    h = heuristic(neighbor, goal)
                    new_f = new_g + h
                    new_path = path + [neighbor]
                    counter += 1
                    heappush(pq, (new_f, counter, neighbor, new_path, new_g))
    
    print(f"✗ A* found no path")
    return None, float('inf'), nodes_explored

# Run A*
astar_path, astar_cost, astar_nodes = a_star(weighted_maze, start, goal)

if astar_path:
    print(f"\nPath length: {len(astar_path)} steps")
    print(f"Total cost: {astar_cost}")
    visualize_weighted_maze(weighted_maze, start, goal, astar_path)

## 6. The Grand Comparison

Let's compare all our search algorithms side by side!

In [ ]:
# For fair comparison, let's also run BFS on the weighted maze
from collections import deque

def bfs_with_cost(maze: np.ndarray, start: Tuple[int, int], goal: Tuple[int, int]):
    """BFS that tracks cost (but doesn't use it for decisions)."""
    queue = deque([(start, [start], 0)])
    visited = set([start])
    nodes_explored = 0
    
    while queue:
        current, path, cost = queue.popleft()
        nodes_explored += 1
        
        if current == goal:
            return path, cost, nodes_explored
        
        for neighbor, step_cost in get_weighted_neighbors(maze, current):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, path + [neighbor], cost + step_cost))
    
    return None, float('inf'), nodes_explored

bfs_path, bfs_cost, bfs_nodes = bfs_with_cost(weighted_maze, start, goal)

# Create comparison table
print("=" * 80)
print("ALGORITHM COMPARISON")
print("=" * 80)
print(f"{'Algorithm':<20} {'Path Cost':<15} {'Nodes Explored':<20} {'Path Length'}")
print("-" * 80)

results = [
    ("BFS", bfs_cost, bfs_nodes, len(bfs_path) if bfs_path else 0),
    ("UCS", ucs_cost, ucs_nodes, len(ucs_path) if ucs_path else 0),
    ("Greedy", greedy_cost, greedy_nodes, len(greedy_path) if greedy_path else 0),
    ("A*", astar_cost, astar_nodes, len(astar_path) if astar_path else 0),
]

for name, cost, nodes, length in results:
    optimal = "⭐" if cost == ucs_cost else "  "
    print(f"{name:<20} {cost:<15.1f} {nodes:<20} {length} steps {optimal}")

print("\n" + "=" * 80)
print("KEY INSIGHTS:")
print("=" * 80)
print(f"✓ Optimal cost: {ucs_cost} (found by UCS and A*)")
print(f"✓ Most efficient: A* (explored only {astar_nodes} nodes)")
print(f"✓ BFS found shortest path in steps, but not lowest cost!")
print(f"✓ Greedy was fast but {'optimal' if greedy_cost == ucs_cost else 'suboptimal'}")

In [ ]:
# Visual comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 14))
algorithms = [
    ("BFS", bfs_path, axes[0, 0]),
    ("UCS (Optimal)", ucs_path, axes[0, 1]),
    ("Greedy", greedy_path, axes[1, 0]),
    ("A* (Optimal + Efficient)", astar_path, axes[1, 1])
]

for name, path, ax in algorithms:
    display = weighted_maze.astype(float)
    display[weighted_maze == 0] = 0.2
    display[weighted_maze == 1] = 1.0
    display[weighted_maze == 2] = 0.5
    
    if path:
        for pos in path[1:-1]:
            display[pos] = 0.35
    
    ax.imshow(display, cmap='RdYlGn_r', interpolation='nearest')
    ax.text(start[1], start[0], 'S', ha='center', va='center',
           fontsize=16, fontweight='bold', color='blue')
    ax.text(goal[1], goal[0], 'G', ha='center', va='center',
           fontsize=16, fontweight='bold', color='green')
    
    # Find cost for this path
    cost = [c for n, c, _, _ in results if n == name.split()[0]][0]
    ax.set_title(f"{name}\nCost: {cost}, Steps: {len(path) if path else 0}")
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
plt.show()

## 7. Heuristic Quality Matters!

Let's see how different heuristics affect A* performance.

In [ ]:
# Test A* with different heuristics
heuristics = [
    ("Manhattan", manhattan_distance),
    ("Euclidean", euclidean_distance),
    ("Chebyshev", chebyshev_distance),
    ("Zero (= UCS)", lambda p1, p2: 0),  # Zero heuristic makes A* behave like UCS
]

print("A* with Different Heuristics:")
print("=" * 70)
print(f"{'Heuristic':<20} {'Cost':<10} {'Nodes':<15} {'Optimal?'}")
print("-" * 70)

optimal_cost = astar_cost

for name, heuristic in heuristics:
    path, cost, nodes = a_star(weighted_maze, start, goal, heuristic)
    is_optimal = "✓" if cost == optimal_cost else "✗"
    print(f"{name:<20} {cost:<10.1f} {nodes:<15} {is_optimal}")

print("\n💡 Key Insight: All admissible heuristics find optimal path!")
print("   Better heuristics explore fewer nodes (more efficient).")

## 8. Exercise: Design Your Own Heuristic

Create a custom heuristic and test it!

In [ ]:
def custom_heuristic(pos1: Tuple[int, int], pos2: Tuple[int, int]) -> float:
    """
    Your custom heuristic here!
    
    Tips:
    - Must be admissible (never overestimate)
    - Higher values = further from goal
    - Can use any calculation you want
    """
    # Example: Manhattan distance
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])
    
    # Try your own!
    # Ideas:
    # - Weighted Manhattan: different weights for x and y
    # - Diagonal distance: accounts for diagonal moves
    # - Octile distance: combination of Manhattan and Euclidean

# Test your heuristic
custom_path, custom_cost, custom_nodes = a_star(weighted_maze, start, goal, custom_heuristic)
print(f"Your heuristic: Cost {custom_cost}, Nodes {custom_nodes}")
print(f"Optimal? {'✓' if custom_cost == optimal_cost else '✗'}")

## 9. Real-World Application: GPS Navigation

Let's simulate a simple GPS-like navigation system.

In [ ]:
# Create a larger city grid with different road types
# 0 = highway (cost 1), 2 = local road (cost 2), 3 = dirt road (cost 4)
city_map = np.array([
    [0, 0, 0, 0, 2, 2, 2, 2],
    [2, 1, 1, 0, 2, 3, 3, 2],
    [2, 1, 1, 0, 2, 3, 3, 2],
    [2, 2, 2, 0, 2, 2, 2, 2],
    [3, 3, 2, 0, 1, 1, 1, 0],
    [3, 3, 2, 0, 1, 1, 1, 0],
    [2, 2, 2, 0, 0, 0, 0, 0],
    [2, 3, 3, 2, 2, 2, 2, 2],
])

TERRAIN_COSTS = {0: 1, 2: 2, 3: 4}  # Updated costs
city_start = (0, 0)
city_goal = (7, 7)

print("Finding route from home (0,0) to destination (7,7)...\n")

# Compare fastest (A*) vs shortest (BFS)
fast_path, fast_cost, fast_nodes = a_star(city_map, city_start, city_goal)
short_path, short_cost, short_nodes = bfs_with_cost(city_map, city_start, city_goal)

print("Route Comparison:")
print(f"Fastest route (A*):    {fast_cost} time units, {len(fast_path)} steps")
print(f"Shortest route (BFS):  {short_cost} time units, {len(short_path)} steps")
print(f"\nA* saved {short_cost - fast_cost:.0f} time units!")

# Visualize both routes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

for ax, path, title in [(ax1, fast_path, "A* (Fastest)"), (ax2, short_path, "BFS (Shortest Steps)")]:
    display = city_map.astype(float) / 4.0  # Normalize for visualization
    if path:
        for pos in path[1:-1]:
            display[pos] = 0.6
    
    ax.imshow(display, cmap='RdYlGn_r')
    ax.text(city_start[1], city_start[0], 'S', ha='center', va='center',
           fontsize=20, fontweight='bold', color='blue')
    ax.text(city_goal[1], city_goal[0], 'G', ha='center', va='center',
           fontsize=20, fontweight='bold', color='green')
    ax.set_title(title)
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
plt.show()

## 10. Key Takeaways

### Algorithm Comparison:

| Algorithm | Optimal? | Efficient? | Uses Cost? | Uses Heuristic? | Best For |
|-----------|----------|-----------|------------|-----------------|----------|
| **BFS** | Steps only | Medium | ✗ | ✗ | Unweighted graphs |
| **UCS** | ✓ | Slow | ✓ | ✗ | Weighted graphs, no heuristic |
| **Greedy** | ✗ | Fast | ✗ | ✓ | Quick solutions, optimality not crucial |
| **A*** | ✓ | ✓ | ✓ | ✓ | **Best overall choice!** |

### When to Use What:

- **BFS**: Simple, unweighted problems (social networks, puzzles)
- **UCS**: Weighted problems, no good heuristic available
- **Greedy**: Need fast results, don't need optimal solution
- **A***: Most real-world problems (GPS, robotics, games)

### Heuristic Guidelines:

1. **Must be admissible** (never overestimate) for optimal A*
2. **Better heuristics** = fewer nodes explored
3. **Common heuristics**: Manhattan (grid, 4-dir), Euclidean (any-dir), Chebyshev (8-dir)
4. **Zero heuristic**: Makes A* behave like UCS

## Next Up

In Lab 3, we'll explore **Adversarial Search**:
- Games with opponents
- Minimax algorithm
- Alpha-Beta pruning
- Building game-playing AI!

## Practice Challenges

1. Implement diagonal movement (8 directions) and adjust heuristics
2. Create a maze where Greedy fails but A* succeeds
3. Design a heuristic that considers terrain type
4. Compare algorithm performance on 50x50 mazes
5. Implement bidirectional A* (search from both start and goal)

Great work! You now understand the core of modern pathfinding algorithms. A* is used in everything from Google Maps to video games! 🎮🗺️